In [1]:
import pandas as pd
from coypu.utils.query import Query
from coypu.utils.credentials import *
from query.queries import *
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

coypu.utils


In [2]:
cmemc_query_imp = Query(client_url_imp, client_id_imp, client_secret_imp , 'oauth')
    
fuseki_infai = Query(fuseki_endpoint_infai, fuseki_user_infai, fuseki_pw_infai, 'basic')

fdquery = Query("http://localhost:5002/sparql", is_fdq=True)

In [3]:
print(query_1_fdq_desc)
print(query_1_fdq)
print('########################### EXAMPLE ##########################\n{}'.format(query_1_fdq_ex))
fdquery.post_query(query_1_fdq_ex)
fdquery.to_save('../../query_results/', filename='query_1_fdq_ex')

Q1: For a given country (?country_code) and indicator (?i), return the value of the indicator and the number of disasters registered per year in that country.
For example:
?country_code = ‘CHN’
?i = <http://worldbank.org/Indicator/EN.ATM.CO2E.KT>

SELECT ?year ?year_dis ?value ?disaster
WHERE {
    ?indicator a wb:AnnualIndicatorEntry .
    ?indicator wb:hasIndicator ?i .
    ?indicator wb:hasCountry ?country .
    ?indicator owl:hasValue ?value .
    ?indicator time:year ?year .
    ?country   dc:identifier ?country_code .

    ?disaster a coy:Disaster .
    ?disaster time:year ?year_dis .
    ?disaster geo:countryCode ?country_code .

}

########################### EXAMPLE ##########################
PREFIX coy: <https://schema.coypu.org/global#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX time: <http://www.w3.org/2006/time#> 

AttributeError: 'Query' object has no attribute 'response'

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

df = pd.read_csv('../../query_results/query_1_fdq_ex.csv', encoding='utf-8')
df.rename(columns=lambda x: x.replace('.value', ''), inplace=True)
try:
    df.to_sql('query_1_fdq_ex', con=engine, if_exists='fail', index=False)
except Exception as e:
    print('Exeception: {}',e)

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

query_1_fdq_ex_sql = """SELECT country_code, year, AVG(value) as carbon_emission, COUNT(disaster) AS no_of_disasters
FROM `query_1_fdq_ex`
WHERE year=year_dis
GROUP By country_code, year
Order By  year ASC
"""

df = pd.read_sql(query_1_fdq_ex_sql, con=engine)

df.to_csv('../../query_results/query_1_fdq_ex_sql.csv', encoding='utf-8', index=False)

In [ ]:
def carbon_emission_group(carbon_emission):
    if carbon_emission/1000000 < 0.25:
        return '0.00-0.25'
    elif carbon_emission/1000000 >= 0.25 and carbon_emission/1000000 < 0.5:
        return '0.25-0.5'
    elif carbon_emission/1000000 >= 0.5 and carbon_emission/1000000 < 0.75:
        return '0.50-0.75'
    else:
        return '> 0.75'
    

df = pd.read_csv('../../query_results/query_1_fdq_ex_sql.csv', encoding='utf-8')
df['carbon_emission(1e6)'] = df['carbon_emission'].apply(carbon_emission_group)

df.head(10)

In [ ]:

fig = px.scatter_geo(df, locations="country_code", color="carbon_emission(1e6)",
                     hover_data=['carbon_emission'], size="no_of_disasters",
                     animation_frame="year",
                     projection="equirectangular", scope='europe')
fig.update_layout(title_text='Carbon emission and the number of disaster events for various countries across years',\
                  title_font_color="red", title_font_family="Times New Roman", title_font_size=25)
fig.write_html('../../query_results/query_1_fdq_ex_sql_new.html')
fig.show()